In [ ]:
%matplotlib qt5
import cv2, time
import numpy as np
import argparse, pprint
from matplotlib import pyplot as plt

import pyrealsense2 as rs

from tools.d415_camera import CameraD415
from src.VBOATS import VBOATS

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
cam = CameraD415()

In [ ]:
vboat = VBOATS()

In [ ]:
extr = cam.get_extrinsics()
intr = cam.get_intrinsics()

pp.pprint(extr)
pp.pprint(intr)

In [ ]:
rgb, depth = cam.read()

tmp = depth/cam.dmax_avg
depth = np.uint8(tmp*255)


plt.figure(0)
plt.imshow(depth)
plt.show()

In [ ]:
vboat.pipeline(depth, threshU1=8,threshU2=20, threshV2=70)

In [ ]:
dispU1, dispU2, compU = vboat.umap_displays()
dispV1, dispV2, compV = vboat.vmap_displays()

plt.figure(2)
# plt.imshow(vboat.vmap_raw)
# plt.imshow(vboat.umap_raw)
plt.imshow(vboat.umap_processed)
plt.imshow(dispU2)
plt.show()

In [ ]:
display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

for ob in vboat.obstacles:
    cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)

In [ ]:
cam.__del__()